In [ ]:
#player index, team may matter in chosing who plays first and who doesn't
#for now I won't use them, but should be considered 
#also role may be important, for now only goalkeeper will be accounted for in the final dataframe
#if a team is away or home depends on the order of the report so team 0 will be home, team 1 away


#it seems some players are missing in the list of players (e.g. Antonino Barilla')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pickle

pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

import umap
from sklearn.decomposition import TruncatedSVD as tsvd

def nearZeroVarDropAuto(df,thresh=0.99):
    vVal=df.var(axis=0).values
    cs=pd.Series(vVal).sort_values(ascending=False).cumsum()
    remove=cs[cs>cs.values[-1]*thresh].index.values
    return df.drop(df.columns[remove],axis=1)

In [ ]:
%run SodaKick_download_functions.ipynb

In [ ]:
df_outfield=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield.hdf',key='pl')
df_team=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Team.hdf',key='pl')
df_vs=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Vs.hdf',key='pl')
df_keeper=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_keeper.hdf',key='pl')
df_fix=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_fix.hdf',key='pl')

# Clean up Outfield

In [ ]:
def clean_outfield(df_outfield):

    df_outfield=df_outfield.fillna(0)

    #df_outfield=df_outfield.drop(['birth_year'],axis=1)
    #df_outfield['age']=df_outfield['age'].apply(lambda x: x.split('-')[0]).astype(int)
    df_outfield=df_outfield.drop(['age'],axis=1)

    df_outfield.nationality = pd.Categorical(df_outfield.nationality)
    df_outfield.position = pd.Categorical(df_outfield.position)
    df_outfield.squad = pd.Categorical(df_outfield.squad)

    nat_dict=dict( enumerate(df_outfield.nationality.cat.categories ) )
    pos_dict=dict( enumerate(df_outfield.position.cat.categories ) )
    squ_dict=dict( enumerate(df_outfield.squad.cat.categories ) )

    #df_outfield['nationality_code']=df_outfield.nationality.cat.codes
    #df_outfield['position_code']=df_outfield.position.cat.codes
    #df_outfield['squad_code']=df_outfield.squad.cat.codes

    df_outfield.set_index('player', inplace=True, drop=True)
    df_outfield=df_outfield[[x for x in df_outfield.columns if '90' not in x]]
    df_outfield=df_outfield.loc[:, ~(df_outfield==0).all(axis=0)]

    #position code category should be ordered from GK, DF to FW
    df_outfull=df_outfield.copy(deep=True)
    df_outfull['nationality_code']=df_outfield.nationality.cat.codes
    df_outfull['position_code']=df_outfield.position.cat.codes
    df_outfull['squad_code']=df_outfield.squad.cat.codes


    df_outfield=df_outfield.drop('nationality',axis=1)
    df_outfield=df_outfield.drop('position',axis=1)
    df_outfield=df_outfield.drop('squad',axis=1)

    #normalize game starts and minutes by the number of games played
    df_outfield[['games_starts','minutes']]=df_outfield[['games_starts','minutes']].div(df_outfield['games'], axis=0)


    #normalize by minutes played
    df_outfield[df_outfield.columns.difference(['nationality_code','position_code','squad_code']+\
                                               ['birth_year', 'games', 'games_starts', 'minutes']+\
                                               [x for x in df_outfield.columns if 'x' in x]+\
                                               [x for x in df_outfield.columns if '_pct' in x])] = \
    df_outfield[df_outfield.columns.difference(['nationality_code','position_code','squad_code']+\
                                                   ['birth_year', 'games', 'games_starts', 'minutes']+\
                                                   [x for x in df_outfield.columns if 'x' in x]+\
                                                   [x for x in df_outfield.columns if '_pct' in x])]\
                                                   .div(df_outfield['minutes'],axis=0)

    df_outfield=df_outfield.astype(float)
    df_outfield['birth_year']=df_outfield['birth_year'].astype(int)
    #print(df_outfield.shape)
    df_outall=df_outfield.copy(deep=True)

    todrop=['players_dribbled_past','passes_short','touches','touches_live_ball','passes_medium','passes_live','passes','passes_received',
    'passes_received_pct','npxg_net', 'carry_progressive_distance', 'npxg','passes_ground','dribbles','pens_att','pressures_mid_3rd',
    'sca_passes_live', 'games_starts', 'passes_long', 'blocked_passes']+['passes_completed','dribbles_vs','pass_targets','touches_mid_3rd','touches_att_3rd','sca_passes_dead','progressive_carries','carries_into_final_third',
          'corner_kicks_out', 'gca_passes_live','passes_into_final_third','tackles_def_3rd','corner_kicks_in']+[x for x in df_outfield.columns if '3rd' in x]+[x for x in df_outfield.columns if 'pct' in x]
    ['shots_on_target_pct','npxg_per_shot','xg_net','passes_pct','passes_progressive_distance','passes_pct_short','passes_pct_medium','passes_pct_long','xa_net',
    'corner_kicks_straight', 'passes_low', 'passes_high',
    'passes_left_foot', 'passes_right_foot', 'passes_head',
    'throw_ins', 'passes_other_body','passes_offsides', 'passes_oob','dribble_tackles_pct','pressure_regain_pct','dribbles_completed_pct',
    'aerials_won_pct']
    
    df_outfield.drop([x for x in todrop if x in df_outfield.columns],axis=1,inplace=True)
    #print(df_outfield.shape)
    
    ### remove duplicate players 
    #except for the year
    age=df_outfield['birth_year'][~df_outfield.index.duplicated(keep='first')]
    df_outfield=df_outfield.sum(level=0)
    df_outfield['birth_year']=age
    #print(df_outfield.shape)
    
    #ugly because of the duplicate players with different positions in different teams
    df_gk_extra=df_outfield.loc[df_outfull[df_outfull['position']=='GK'].index.unique()]
    df_outfield=df_outfield.loc[df_outfull[df_outfull['position']!='GK'].index.unique()]
    df_outfull=df_outfull.loc[df_outfield.index]

    
    
    return df_outfield, df_outall, df_gk_extra
    

In [ ]:
# correlation removal, now skipping
"""def corr_pairs(df):
    correlated=[] # Set of all the names of deleted columns
    corr_matrix = df.corr().abs()
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1,len(corr_matrix.columns)):
            correlated.append((df.columns[i],df.columns[j],corr_matrix.iloc[i, j]))
    
    return pd.DataFrame(correlated, columns=['col1','col2','correlation']).sort_values('correlation',ascending=False)

corr_cols=corr_pairs(df_outfield)
sns.distplot(corr_cols['correlation'])

def iteremove_corr(df,thr, limit=-1):
    
    tmp=df.copy(deep=True)
    print(tmp.shape[1], end=' ')
    step=0
    
    while True and step!=limit:
        
        correlated=[] # Set of all the names of deleted columns
        corr_matrix = tmp.corr().abs()
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1,len(corr_matrix.columns)):
                correlated.append((tmp.columns[i],tmp.columns[j],corr_matrix.iloc[i, j]))

        corr_cols=pd.DataFrame(correlated, columns=['col1','col2','correlation']).sort_values('correlation',ascending=False)

        if corr_cols['correlation'].iloc[0]<=thr:
            break
            
        tmp.drop(corr_cols[corr_cols['correlation']>thr]['col2'].values,axis=1, inplace=True)
        print(tmp.shape[1], end=' ')
        step+=1
    
    return tmp
        
df_outfield_nc=iteremove_corr(df_outfield,.75, limit=1)
corr_cols=corr_pairs(df_outfield_nc)

sns.distplot(corr_cols['correlation'])"""

#tsvd on single, now skipping will be done at the end with everything

"""ts=tsvd(n_components=int(df_outfield.shape[1])-1, random_state=32)
ts.fit_transform(df_outall)
tsvd_var_ratios = ts.explained_variance_ratio_

def select_n_components(var_ratio, goal_var: float) -> int:

    total_variance = 0.0
    n_components = 0

    for explained_variance in var_ratio:
        total_variance += explained_variance        
        n_components += 1
        if total_variance >= goal_var:
            break

    return n_components

# Run function
num_comp=select_n_components(tsvd_var_ratios, 0.99)
print(num_comp)
ts=tsvd(n_components=num_comp, random_state=32)
df_outtsvd=pd.DataFrame(ts.fit_transform(df_outall),index=df_outall.index)"""

#mapping

"""mapping=umap.UMAP(metric='cosine', n_components=2, min_dist=0.0, spread=1, n_neighbors=int(np.sqrt(df_outfield_nc.shape[0])), \
                                n_epochs=2500, learning_rate=0.05, \
                                verbose=False, random_state=32)
mapping.fit(df_outfield_nc.astype(float).values)
mh=pd.DataFrame(mapping.transform(df_outfield_nc.values), index=df_outfield_nc.index)

for c in df_outfull.position_code.unique():
    plt.scatter(mh[0][df_outfull.position_code==c],mh[1][df_outfull.position_code==c],label=pos_dict[c])
    
plt.legend()


sel_cut=nearZeroVarDropAuto(df_outfield, .999)
print(sel_cut.shape)

mapping=umap.UMAP(metric='cosine', n_components=2, min_dist=0.0, spread=1, n_neighbors=int(np.sqrt(sel_cut.shape[0])), \
                                n_epochs=2500, learning_rate=0.05, \
                                verbose=False, random_state=32)
mapping.fit(sel_cut.astype(float).values)
mh=pd.DataFrame(mapping.transform(sel_cut.values), index=sel_cut.index)

for c in df_outfull.position_code.unique():
    plt.scatter(mh[0][df_outfull.position_code==c],mh[1][df_outfull.position_code==c],label=pos_dict[c])
    
plt.legend()

sel_cut=df_outtsvd
mapping=umap.UMAP(metric='cosine', n_components=2, min_dist=0.0, spread=1, n_neighbors=int(np.sqrt(sel_cut.shape[0])), \
                                n_epochs=2500, learning_rate=0.05, \
                                verbose=False, random_state=32)
mapping.fit(sel_cut.astype(float).values)
mh=pd.DataFrame(mapping.transform(sel_cut.values), index=sel_cut.index)

for c in df_outfull.position_code.unique():
    plt.scatter(mh[0][df_outfull.position_code==c],mh[1][df_outfull.position_code==c],label=pos_dict[c])
    
plt.legend()"""

#df_outfield.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield_clean.hdf',key='pl')
#df_outall.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield_allfeat.hdf',key='pl')
#df_outtsvd.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield_tsvd.hdf',key='pl')

# Clean Up Keeper

In [ ]:
def clean_keeper(df_keeper, df_gk_extra):
    df_keeper=df_keeper.fillna(0)

    #df_keeper=df_keeper.drop(['birth_year'],axis=1)
    #df_keeper['age']=df_keeper['age'].apply(lambda x: x.split('-')[0]).astype(int)
    df_keeper=df_keeper.drop(['age'],axis=1)
    df_keeper.columns=[x[:-3] if x.endswith('_gk') else x for x in df_keeper.columns]

    df_keeper.nationality = pd.Categorical(df_keeper.nationality)
    df_keeper.position = pd.Categorical(df_keeper.position)
    df_keeper.squad = pd.Categorical(df_keeper.squad)

    nat_dict=dict( enumerate(df_keeper.nationality.cat.categories ) )
    pos_dict=dict( enumerate(df_keeper.position.cat.categories ) )
    squ_dict=dict( enumerate(df_keeper.squad.cat.categories ) )

    df_keeper.set_index('player', inplace=True, drop=True)
    df_keeper=df_keeper[[x for x in df_keeper.columns if '90' not in x]]
    df_keeper=df_keeper.loc[:, ~(df_keeper==0).all(axis=0)]



    #position code category should be ordered from GK, DF to FW
    df_keefull=df_keeper.copy(deep=True)
    df_keefull['nationality_code']=df_keeper.nationality.cat.codes
    df_keefull['position_code']=df_keeper.position.cat.codes
    df_keefull['squad_code']=df_keeper.squad.cat.codes


    df_keeper=df_keeper.drop('nationality',axis=1)
    df_keeper=df_keeper.drop('position',axis=1)
    df_keeper=df_keeper.drop('squad',axis=1)

    #normalize game starts and minutes by the number of games played
    df_keeper[['games_starts','minutes']]=df_keeper[['games_starts','minutes']].div(df_keeper['games'], axis=0)


    #normalize by minutes played
    df_keeper[df_keeper.columns.difference(['nationality_code','position_code','squad_code']+\
                                               ['birth_year', 'games', 'games_starts', 'minutes']+\
                                               [x for x in df_keeper.columns if 'x' in x]+\
                                               [x for x in df_keeper.columns if '_pct' in x])] = \
        df_keeper[df_keeper.columns.difference(['nationality_code','position_code','squad_code']+\
                                                   ['birth_year', 'games', 'games_starts', 'minutes']+\
                                                   [x for x in df_keeper.columns if 'x' in x]+\
                                                   [x for x in df_keeper.columns if '_pct' in x])]\
                                                   .div(df_keeper['minutes'],axis=0)

    df_keeper=df_keeper.astype(float)
    df_keeper['birth_year']=df_keeper['birth_year'].astype(int)
    #print(df_keeper.shape)
    df_keeall=df_keeper.copy(deep=True)

    todrop=['games_starts','save_pct','clean_sheets_pct','passes_throws','passes_launched','wins','losses','draws']+[x for x in df_keeper.columns if 'pct' in x]
    df_keeper.drop([x for x in todrop if x in df_keeper.columns],axis=1,inplace=True)
    

    age=df_keeper['birth_year'][~df_keeper.index.duplicated(keep='first')]
    df_keeper=df_keeper.sum(level=0)
    df_keeper['birth_year']=age
    #print(df_keeper.shape)
    
    df_keeper=pd.concat([df_keeper,df_gk_extra],axis=1)
    df_keeper = df_keeper.loc[:,~df_keeper.columns.duplicated()]
    df_keeper=df_keeper.loc[:, ~(df_keeper==0).all(axis=0)]
    
    return df_keeper, df_keeall

In [ ]:
#df_keeper.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Keeper_clean.hdf',key='pl')
#df_keeall.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Keeper_allfeat.hdf',key='pl')
#df_keetsvd.to_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Keeper_tsvd.hdf',key='pl')

In [ ]:
#df_outfield=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield_clean.hdf',key='pl')
#df_keeper=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Keeper_clean.hdf',key='pl')

def clean_players(df_outfield, df_keeper):
    
    df_outfield, df_outfield_all, df_gk_extra = clean_outfield(df_outfield)
    df_keeper, df_keall = clean_keeper(df_keeper, df_gk_extra)
    
    df_outfield['keeper']=0
    df_keeper['keeper']=1

    df_players=pd.concat([df_outfield,df_keeper],axis=0).fillna(0)
    
    return df_players

# Build Team from Report

In [2]:
def build_team_from_lineup(lineup, df_players):

    df_players=df_players.loc[lineup.index.intersection(df_players.index)]

    df_players=df_players.append(pd.DataFrame(np.nan, columns=df_players.columns, index=lineup.index.difference(df_players.index)))
    df_players=df_players.loc[lineup.index].fillna(0)
    
    df_players['bench']=lineup['bench']
    df_players['team']=lineup['team']
    #missing=df_players[df_players.isna().any(axis=1)].index

    df_players = df_players[~df_players.index.duplicated(keep='first')]
    #df_players = df_players.fillna(0)
    
    return df_players, lineup.index.difference(df_players.index)

def fillto50(lineup):
    team1=lineup[lineup['team']==0]
    if len(team1)<25:
        team1=team1.append(pd.DataFrame([[0,1,lineup['formation'].iloc[0]]]*(25-team1.shape[0]), 
                                        columns=team1.columns, 
                                        index=['dummy1'+str(x) for x in range((25-team1.shape[0]))]))

    team2=lineup[lineup['team']==1]
    if len(team2)<25:
        team2=team2.append(pd.DataFrame([[0,1,lineup['formation'].iloc[0]]]*(25-team2.shape[0]), 
                                        columns=team2.columns, 
                                        index=['dummy2'+str(x) for x in range((25-team2.shape[0]))]))

    return pd.concat([team1,team2],axis=0)

#df_players.drop(['wins','draws','losses'], axis=1, inplace=True)

In [3]:
def clean_report(match, df_players):
    
        team1,team2,gk1,gk2,shots,lineup=get_report_data(path=match)
        teams=pd.concat([team1,team2],axis=0).reset_index(drop=True)
        gks=pd.concat([gk1,gk2],axis=0).reset_index(drop=True)

        teams.set_index('player',drop=True,inplace=True)
        gks.set_index('player',drop=True,inplace=True)
        lineup.set_index('player',drop=True,inplace=True)

        lineup['bench']=lineup['bench'].astype(int)
        lineup.team=(lineup.team==lineup.team.unique()[1]).astype(int)
        lineup=fillto50(lineup)
                
        teams=teams[['minutes','goals','assists','cards_yellow','cards_red','own_goals']]
        gks.columns=[x[:-3] if x.endswith('_gk') else x for x in gks.columns]
        gks=gks[['goals_against','saves']]
        #results=pd.concat([teams,gks],axis=1).loc[lineup.index].fillna(0)
        
        results=pd.concat([teams,gks],axis=1)
        results=results.append(pd.DataFrame(np.nan, columns=results.columns, index=lineup.index.difference(results.index)))
        results=results.loc[lineup.index].fillna(0)
        
        inp, missing=build_team_from_lineup(lineup, df_players)

        #possibly redundant
        results.loc[missing]=0
        
        return inp, results

# tsvd all 

In [18]:
df_players=[]
df_fixs=[]
for series in ['SA2021','SA1920','BL2021','BL1920','LU2021','LU1920','PL2021','PL1920','LL2021','LL1920']+\
              ['SA1819','SA1718','BL1819','BL1718','LU1819','LU1718','PL1819','PL1718','LL1819','LL1718']:

    df_outfield = pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/'+series+'_Outfield.hdf')
    df_keeper = pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/'+series+'_Keeper.hdf')
    cp = clean_players(df_outfield, df_keeper)
    if 'gca_og_for' in cp:
        #this is available only for the last year
        cp.drop(['gca_og_for'],axis=1,inplace=True) 
    if 'corner_kicks_straight' in cp:
        #this is available only for the last year
        cp.drop(['corner_kicks_straight'],axis=1,inplace=True) 
    df_players.append(cp.replace(np.inf,0))#.fillna(0).astype(float))
    
    df_fixs.append(pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/'+series+'_Fix.hdf'))
   
allpl = pd.concat(df_players, axis=0).fillna(0).astype(float)
allpl.drop(['keeper'], axis=1, inplace=True)

In [13]:
ts=tsvd(n_components=int(allpl.shape[1])-1, random_state=32)
ts.fit(allpl)
tsvd_var_ratios = ts.explained_variance_ratio_

def select_n_components(var_ratio, goal_var: float) -> int:

    total_variance = 0.0
    n_components = 0

    for explained_variance in var_ratio:
        total_variance += explained_variance        
        n_components += 1
        if total_variance >= goal_var:
            break

    return n_components

# Run function
num_comp=select_n_components(tsvd_var_ratios, 0.9999)
print(num_comp)
ts=tsvd(n_components=num_comp, random_state=32)
ts.fit(allpl)

df_players_ts=[]
for i,play in enumerate(df_players):
    #print(ts.transform(play))
    #print([x for x in allpl.columns if x not in play.columns])
    kp = play['keeper']
    play = play.drop(['keeper'],axis=1)
    try:
        ts.transform(play)
    except:
        print(i)
    df_players_ts.append(pd.DataFrame(ts.transform(play), index=play.index))
    df_players_ts[-1]['keeper'] = kp

20


In [17]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]

for i,series in enumerate(['SA2021','SA1920','BL2021','BL1920',
                           'LU2021','LU1920','PL2021','PL1920',
                           'LL2021','LL1920']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:

            inp, results = clean_report(match, df_players_ts[i])

            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())

        except:

            pass

    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2a.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2a.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)

SA2021
357
SA1920
737
BL2021
1024
BL1920
1330
LU2021
1689
LU1920
1968
PL2021
2315
PL1920
2693
LL2021
3043
LL1920
3414


In [ ]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]

for i,series in enumerate(['SA1819','SA1718','BL1819','BL1718',
                           'LU1819','LU1718','PL1819','PL1718',
                           'LL1819','LL1718']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:
            inp, results = clean_report(match, df_players_ts[i])

            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())
            
        except:
        
            print(match)

    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2b.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2b.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)

SA1819
/en/matches/f68d48b6/Internazionale-Benevento-January-30-2021-Serie-A
357
SA1718


In [21]:
def revert_output(output,lineup):

    reframe=pd.DataFrame(output.reshape(50,8), index=lineup.index,
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
    byteamframe=pd.concat([reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==0].index]].sum(axis=0),
                        reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==1].index]].sum(axis=0)], axis=1).T
    
    return reframe, byteamframe[byteamframe.columns[1:]]

# old but gold

# build dataset of games

In [141]:
inputs=[]
outputs=[]
for match in df_fix['match_report'].values:
    
    try:

        inp, results = clean_report(match, df_players)

        inputs.append(inp.astype(float).values.flatten())
        outputs.append(results.astype(float).values.flatten())
    
    except:
        
        pass
        
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_inp.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_out.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc

# tsvd

In [40]:
df_outfield=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Outfield_clean.hdf',key='pl')
df_keeper=pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/SA2021_Keeper_clean.hdf',key='pl')

df_players=pd.concat([df_outfield,df_keeper],axis=0).fillna(0)
#df_players.drop(['wins','draws','losses'], axis=1, inplace=True)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [41]:
ts=tsvd(n_components=int(df_players.shape[1])-1, random_state=32)
ts.fit_transform(df_players)
tsvd_var_ratios = ts.explained_variance_ratio_


def select_n_components(var_ratio, goal_var: float) -> int:

    total_variance = 0.0
    n_components = 0

    for explained_variance in var_ratio:
        total_variance += explained_variance        
        n_components += 1
        if total_variance >= goal_var:
            break

    return n_components

# Run function
num_comp=select_n_components(tsvd_var_ratios, 0.9999)
print(num_comp)
ts=tsvd(n_components=num_comp, random_state=32)
df_players=pd.DataFrame(ts.fit_transform(df_players),index=df_players.index)

18


In [44]:
df_players['keeper']=0
df_players['keeper'].loc[df_keeper.index]=1
df_players.shape

(587, 19)

In [45]:
inputs=[]
outputs=[]
lineups=[]
for match in df_fix['match_report'].values:
    
    try:

        team1,team2,gk1,gk2,shots,lineup=get_report_data(path=match)
        teams=pd.concat([team1,team2],axis=0).reset_index(drop=True)
        gks=pd.concat([gk1,gk2],axis=0).reset_index(drop=True)

        teams.set_index('player',drop=True,inplace=True)
        gks.set_index('player',drop=True,inplace=True)
        lineup.set_index('player',drop=True,inplace=True)

        lineup['bench']=lineup['bench'].astype(int)
        lineup.team=(lineup.team==lineup.team.unique()[1]).astype(int)
        lineup=fillto50(lineup)

        lineups.append(lineup)
        teams=teams[['minutes','goals','assists','cards_yellow','cards_red','own_goals']]
        gks.columns=[x[:-3] if x.endswith('_gk') else x for x in gks.columns]
        gks=gks[['goals_against','saves']]
        results=pd.concat([teams,gks],axis=1).loc[lineup.index].fillna(0)

        inp, missing=build_team_from_lineup(lineup, df_players=df_players)
        
        #possibly redundant
        results.loc[missing]=0

        inputs.append(inp.astype(float).values.flatten())
        outputs.append(results.astype(float).values.flatten())
    
    except:
        
        pass
        
inputs=np.array(inputs)
outputs=np.array(outputs)

with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_inp_tsvd.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_out_tsvd.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_lineups.pkl', 'wb') as pk:
    pickle.dump(lineups,pk)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc

In [5]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/SA2021_out.pkl', 'rb') as pk:
    out=pickle.load(pk)

In [46]:
reframe, byteam= revert_output(out[0], lineups[0])

In [47]:
byteam

,goals,assists,cards_yellow,cards_red,own_goals,goals_against,saves
0,1.0,1.0,3.0,0.0,0.0,0.0,3.0
1,0.0,0.0,1.0,0.0,0.0,1.0,5.0


In [ ]:
reframe

In [ ]:
from sklearn.metrics import mean_squared_error


# allplayers by year

In [12]:
df_players=[]
allpl_year=[]
    
for year in ['1718','1819','1920','2021']:
    print(year)
    
    for league in ['SA','BL','LU','PL','LL']:
        
        print(league)
        series=league+year
        df_outfield = pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/'+series+'_Outfield.hdf')
        df_keeper = pd.read_hdf('/Users/federico comitani/GitHub/sodakick/data/'+series+'_Keeper.hdf')
        cp = clean_players(df_outfield, df_keeper)
        if 'gca_og_for' in cp:
            #this is available only for the last year
            cp.drop(['gca_og_for'],axis=1,inplace=True) 
        if 'corner_kicks_straight' in cp:
            #this is available only for the last year
            cp.drop(['corner_kicks_straight'],axis=1,inplace=True) 

        df_players.append(cp.replace(np.inf,0))#.fillna(0).astype(float))

    
    ay = pd.concat(df_players, axis=0).fillna(0).astype(float)
    
    age=ay['birth_year'][~ay.index.duplicated(keep='first')]
    ay=ay.sum(level=0)
    ay['birth_year']=age
    
    allpl_year.append(ay)    
 

1718
SA
BL
LU
PL
LL
1819
SA
BL
LU
PL
LL
1920
SA
BL
LU
PL
LL
2021
SA
BL
LU
PL
LL


In [13]:
for i in range(len(allpl_year)):
    for j in range(len(allpl_year)):
        if i!=j:
            missing=allpl_year[j].index.difference(allpl_year[i].index)
            missingDf=pd.DataFrame(np.nan,columns=allpl_year[j].columns,index=missing)
            missingDf['birth_year']=allpl_year[j]['birth_year'].loc[missing]
            allpl_year[i]=allpl_year[i].append(missingDf)
            
for i in range(len(allpl_year)):
    allpl_year[i]=allpl_year[i].sort_index()
    
#fillna if you want to keep them as separate channels

In [14]:
#merge them with weights
from functools import reduce
import copy
allpl_yr_nu=copy.deepcopy(allpl_year)

allpl_weighted=[]
while len(allpl_yr_nu)>0:
    
    current=allpl_yr_nu.pop()
    columns=current.columns
    weight=1.0
    
    by=current['birth_year']
    current.drop(['birth_year'],axis=1,inplace=True)
    
    div=pd.Series([1]*current.shape[0], index=current.index)
    
    for i in range(len(allpl_yr_nu))[::-1]:
    
        weight=weight/2
        
        to_add=allpl_yr_nu[i]*weight
        to_add.drop(['birth_year'],axis=1,inplace=True)

        notmissing=np.where(~to_add.isnull().all(1))
        div.iloc[notmissing]+=weight

        to_add=to_add.fillna(0)
        

        current=current.add(to_add)
        
    current=current.divide(div,axis=0)
    current.dropna(how='all',axis=0,inplace=True)

    current['birth_year']=by.loc[current.index]
    
    allpl_weighted.append(current[columns])
    
allpl_weighted=allpl_weighted[::-1]

In [15]:
allpl=pd.concat(allpl_weighted, axis=0)
allpl.drop(['keeper'], axis=1, inplace=True)

ts=tsvd(n_components=int(allpl.shape[1])-1, random_state=32)
ts.fit(allpl)
tsvd_var_ratios = ts.explained_variance_ratio_

def select_n_components(var_ratio, goal_var: float) -> int:

    total_variance = 0.0
    n_components = 0

    for explained_variance in var_ratio:
        total_variance += explained_variance        
        n_components += 1
        if total_variance >= goal_var:
            break

    return n_components

# Run function
num_comp=select_n_components(tsvd_var_ratios, 0.9999)
print(num_comp)
ts=tsvd(n_components=num_comp, random_state=32)
ts.fit(allpl)

allpl_weighted_ts=[]
for i,play in enumerate(allpl_weighted):
    #print(ts.transform(play))
    #print([x for x in allpl.columns if x not in play.columns])
    kp = play['keeper']
    play = play.drop(['keeper'],axis=1)
    try:
        ts.transform(play)
    except:
        print(i)
    allpl_weighted_ts.append(pd.DataFrame(ts.transform(play), index=play.index))
    allpl_weighted_ts[-1]['keeper'] = kp

15


In [233]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/allplayers_weighted.pkl', 'wb') as pk:
    pickle.dump(allpl_weighted,pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/allplayers_weighted_tsvd.pkl', 'wb') as pk:
    pickle.dump(allpl_weighted_ts,pk)

In [16]:
players_year_dict={'1718': allpl_weighted_ts[0],
                   '1819': allpl_weighted_ts[1],
                   '1920': allpl_weighted_ts[2],
                   '2021': allpl_weighted_ts[3]}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]

for i,series in enumerate(['SA2021','SA1920','BL2021','BL1920','LU2021','LU1920','PL2021','PL1920','LL2021','LL1920']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:

            inp, results = clean_report(match, players_year_dict[series[2:]])

            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())

        except:

            pass

    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_a.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_a.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)

SA2021
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(40, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(39, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(45, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(46, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(41, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(40, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(41, 16)
(50, 16)
(50, 16)
(50, 18)
(41, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(42, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50, 16)
(50, 18)
(44, 16)
(50, 16)
(50, 16)
(50, 18)
(41, 16)
(50, 16)
(50, 16)
(50, 18)
(43, 16)
(50, 16)
(50

In [48]:
import warnings
warnings.filterwarnings('ignore')

inputs=[]
outputs=[]

for i,series in enumerate(['SA1819','SA1718','BL1819','BL1718','LU1819','LU1718','PL1819','PL1718','LL1819','LL1718']):
    
    print(series)
    
    for match in df_fixs[i]['match_report'].values:

        try:

            inp, results = clean_report(match, players_year_dict[series[2:]])

            inputs.append(inp.astype(float).values.flatten())
            outputs.append(results.astype(float).values.flatten())

        except:

            pass

    print(len(inputs))
    
inputs=np.array(inputs)
outputs=np.array(outputs)


with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_b.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_b.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)

SA1819
357
SA1718
737
BL1819
1024
BL1718
1330
LU1819
1689
LU1718
1968
PL1819
2315
PL1718
2693
LL1819
3045
LL1718
3422


In [32]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_a.pkl', 'rb') as pk:
    inputs=pickle.load(pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_a.pkl', 'rb') as pk:
    outputs=pickle.load(pk)
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp.pkl', 'rb') as pk:
    inputs2=pickle.load(pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out.pkl', 'rb') as pk:
    outputs2=pickle.load(pk)

In [46]:
inputs=np.concatenate([inputs,inputs2])
outputs=np.concatenate([outputs,outputs2])

In [47]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp.pkl', 'wb') as pk:
    pickle.dump(inputs,pk)

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out.pkl', 'wb') as pk:
    pickle.dump(outputs,pk)